### Johansen Test

The Johansen test is used to test for cointegration among more than two variables, extending the idea of cointegration to a system of multiple time series. This involves generalizing a price variable equation to matrices and vectors, and the test determines whether these price series are cointegrated by calculating the rank (r) of a matrix Λ. If Λ has a rank of zero, no cointegration exists.

The test computes two statistics, the trace and eigen statistics, based on eigenvector decomposition of Λ. These statistics help test null hypotheses about the rank of Λ, with rejection indicating cointegration. The test provides critical values, and if all null hypotheses are rejected, it suggests full cointegration (r = n).

The test results also provide eigenvectors, which can be used to determine hedge ratios for forming a stationary portfolio. In the example provided, the Johansen test is applied to three ETFs (EWA, EWC, and IGE) to explore cointegrating relationships and build a mean-reverting portfolio.

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm

# Function to download ETF data
def download_data(etfs, start_date, end_date):
    data = yf.download(etfs, start=start_date, end=end_date)['Adj Close']
    return data

# Step 1: Download EWA, EWC, and IGE price series
etfs = ['EWA', 'EWC', 'IGE']
start_date = '2010-01-01'
end_date = '2023-01-01'

data = download_data(etfs, start_date, end_date)

# Step 2: Johansen test function
def johansen_test(data, det_order=0, k_ar_diff=1):
    # Perform the Johansen cointegration test
    jres = coint_johansen(data, det_order, k_ar_diff)
    
    # Eigenvalues and eigenvectors
    eigenvalues = jres.eig
    eigenvectors = jres.evec

    print("\nJohansen Test Results:")
    print("Eigenvalues:\n", eigenvalues)
    print("\nEigenvectors:\n", eigenvectors)
    
    print("\nTest Statistics:")
    print("\nTrace Statistic:")
    print(pd.DataFrame({
        "Statistic": jres.lr1,
        "Critical Value (90%)": jres.cvt[:, 0],
        "Critical Value (95%)": jres.cvt[:, 1],
        "Critical Value (99%)": jres.cvt[:, 2]
    }))
    
    print("\nEigen Statistic:")
    print(pd.DataFrame({
        "Statistic": jres.lr2,
        "Critical Value (90%)": jres.cvm[:, 0],
        "Critical Value (95%)": jres.cvm[:, 1],
        "Critical Value (99%)": jres.cvm[:, 2]
    }))
    
    return jres

# Step 3: Run the Johansen test
johansen_result = johansen_test(data)

# Step 4: Form the stationary portfolio from the first eigenvector
# The eigenvector provides the hedge ratio for the ETFs
eigenvector_1 = johansen_result.evec[:, 0]
print("\nFirst Eigenvector (Hedge Ratios):", eigenvector_1)

# Step 5: Calculate the portfolio value (yport)
yport = np.dot(data, eigenvector_1)

# Step 6: Estimate the half-life of mean reversion
def estimate_half_life(yport):
    yport_lagged = np.roll(yport, 1)[1:]
    delta_yport = np.diff(yport)
    
    X = sm.add_constant(yport_lagged)  # Add constant for intercept
    model = sm.OLS(delta_yport, X).fit()
    
    # Lambda is the coefficient of yport_lagged
    lambda_value = model.params[1]
    
    # Calculate half-life
    half_life = -np.log(2) / lambda_value
    return half_life

# Step 7: Calculate half-life
half_life = estimate_half_life(yport)
print("\nHalf-life of Mean Reversion:", half_life)


[*********************100%%**********************]  3 of 3 completed

Johansen Test Results:
Eigenvalues:
 [0.00528164 0.00284827 0.00089533]

Eigenvectors:
 [[ 0.57628246 -0.82069854  0.02446138]
 [-0.42937842  0.48993941 -0.24710707]
 [-0.10839989 -0.15620322  0.17105366]]

Test Statistics:

Trace Statistic:
   Statistic  Critical Value (90%)  Critical Value (95%)  Critical Value (99%)
0  29.572889               27.0669               29.7961               35.4628
1  12.256145               13.4294               15.4943               19.9349
2   2.929028                2.7055                3.8415                6.6349

Eigen Statistic:
   Statistic  Critical Value (90%)  Critical Value (95%)  Critical Value (99%)
0  17.316744               18.8928               21.1314               25.8650
1   9.327117               12.2971               14.2639               18.5200
2   2.929028                2.7055                3.8415                6.6349

First Eigenvector (Hedge Ratios): [ 0